# Import Library

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
from unicodedata import normalize
from scipy.io import wavfile
from collections import defaultdict, Counter
from scipy import signal
import numpy as np
import librosa
import sklearn
import random
from keras.layers import Dense
from keras import Input
from keras import Model
from keras.utils import to_categorical
from keras.layers import Dense, TimeDistributed, Dropout, Bidirectional, GRU, BatchNormalization, Activation, LeakyReLU, LSTM, Flatten, RepeatVector, Permute, Multiply, Conv2D, MaxPooling2D

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loda Data

In [ ]:
DATA_DIR_TRAIN = '/content/drive/My Drive/leanai/ML/STT/dataset/train/'
DATA_DIR_TEST = '/content/drive/My Drive/leanai/ML/STT/dataset/test/'

# Preprocessing

In [ ]:
train_X = []
train_spectrograms = []
train_mel_spectograms = []
train_mfccs = []
train_y = []

test_X = []
test_spectrograms = []
test_mel_spectograms = []
test_mfccs = []
test_y = []

pad1d = lambda a, i: a[0: i] if a.shape[0] > i else np.hstack((a, np.zeros(i-a.shape[0])))
pad2d = lambda a, i: a[:, 0:i] if a.shape[1] > i else np.hstack((a, np.zeros((a.shape[0], i-a.shape[1]))))

dataset0 = []
dataset1 = []

test_dataset = []
train_dataset = []

frame_length = 0.025
frame_stride = 0.001

In [ ]:
for filename in os.listdir(DATA_DIR_TRAIN):
  filename = normalize('NFC', filename)
  try:
    if '.wav' not in filename:
      continue

    wav, sr = librosa.load(DATA_DIR_TRAIN + filename, sr=16000)

    mfcc = librosa.feature.mfcc(y=wav)
    padded_mfcc = pad2d(mfcc, 40)

    if filename[0] == '외':
      train_dataset.append((padded_mfcc, 1))
    else:
      train_dataset.append((padded_mfcc, 0))
  except Exception as e:
    print(filename, e)
    raise

for filename in os.listdir(DATA_DIR_TEST):
  filename = normalize('NFC', filename)
  try:
    if '.wav' not in filename:
      continue

    wav, sr = librosa.load(DATA_DIR_TEST + filename, sr=16000)

    input_nfft = int(round(sr*frame_length))
    input_stride = int(round(sr*frame_stride))

    mfcc = librosa.feature.mfcc(y=wav)
    padded_mfcc = pad2d(mfcc, 40)

    if filename[0] == '외':
      test_dataset.append((padded_mfcc, 1))
    else:
      test_dataset.append((padded_mfcc, 0))
  except Exception as e:
    print(filename, e)
    raise

In [ ]:
random.shuffle(test_dataset)
random.shuffle(train_dataset)

train_mfccs = [a for (a,b) in train_dataset]
train_y = [b for (a,b) in train_dataset]

test_mfccs = [a for (a,b) in test_dataset]
test_y = [b for (a,b) in test_dataset]

train_mfccs = np.array(train_mfccs)
train_y = to_categorical(np.array(train_y))

test_mfccs = np.array(test_mfccs)
test_y = to_categorical(np.array(test_y))

print('train_mfccs:', train_mfccs.shape)
print('train_y:', train_y.shape)

print('test_mfccs:', test_mfccs.shape)
print('test_y:', test_y.shape)

train_mfccs: (3145, 20, 40)
train_y: (3145, 2)
test_mfccs: (200, 20, 40)
test_y: (200, 2)


# Training

In [ ]:
train_X_ex = np.expand_dims(train_mfccs, -1)
test_X_ex = np.expand_dims(test_mfccs, -1)
print('train X shape:', train_X_ex.shape)
print('test X shape:', test_X_ex.shape)

train X shape: (3145, 20, 40, 1)
test X shape: (200, 20, 40, 1)


In [ ]:
ip = Input(shape=train_X_ex[0].shape)

m = Conv2D(32, kernel_size=(4,4), activation='relu')(ip)
m = MaxPooling2D(pool_size=(4,4))(m)

m = Conv2D(32*2, kernel_size=(4,4), activation='relu')(ip)
m = MaxPooling2D(pool_size=(4,4))(m)

m = Conv2D(32*3, kernel_size=(4,4), activation='relu')(ip)
m = MaxPooling2D(pool_size=(4,4))(m)

m = Flatten()(m)

m = Dense(32)(m)
m = Activation("relu")(m)

op = Dense(2, activation='softmax')(m)

model = Model(ip, op)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20, 40, 1)]       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 17, 37, 96)        1632      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 4, 9, 96)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 3456)              0         
                                                                 
 dense (Dense)               (None, 32)                110624    
                                                                 
 activation (Activation)     (None, 32)                0         
                                                             

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(train_X_ex,
                    train_y,
                    epochs=100,
                    batch_size=32,
                    verbose=1,
                    validation_data=(test_X_ex, test_y))

Epoch 1/100
99/99 [==============================] - 4s 7ms/step - loss: 0.9052 - accuracy: 0.8283 - val_loss: 0.3151 - val_accuracy: 0.8550
Epoch 2/100
99/99 [==============================] - 0s 4ms/step - loss: 0.2567 - accuracy: 0.8865 - val_loss: 0.2169 - val_accuracy: 0.9150
Epoch 3/100
99/99 [==============================] - 0s 4ms/step - loss: 0.2022 - accuracy: 0.9205 - val_loss: 0.3056 - val_accuracy: 0.8850
Epoch 4/100
99/99 [==============================] - 0s 4ms/step - loss: 0.1696 - accuracy: 0.9316 - val_loss: 0.1577 - val_accuracy: 0.9250
Epoch 5/100
99/99 [==============================] - 0s 4ms/step - loss: 0.1341 - accuracy: 0.9482 - val_loss: 0.2906 - val_accuracy: 0.8950
Epoch 6/100
99/99 [==============================] - 0s 4ms/step - loss: 0.1173 - accuracy: 0.9539 - val_loss: 0.2333 - val_accuracy: 0.9100
Epoch 7/100
99/99 [==============================] - 0s 4ms/step - loss: 0.1132 - accuracy: 0.9552 - val_loss: 0.1734 - val_accuracy: 0.9250
Epoch 8/100
9

# model save

In [ ]:
model.save('/content/drive/My Drive/leanai/ML/STT/model/binary_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
